In [41]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import depth2fgpcd, depth2fgpcd_top

In [42]:
env = 'shirt'
view = 4
dir_path = f'ptcl_data/{env}/view_{str(view)}'
dir_path

'ptcl_data/shirt/view_4'

In [43]:
raw_obs = np.load(os.path.join(dir_path, 'obs.npy'))
raw_obs.shape

(720, 720, 5)

In [44]:
camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy'))
camera_intrinsic_params

array([869.11683083, 869.11683083, 360.        , 360.        ])

In [45]:
camera_ext_matrix = np.load(os.path.join(dir_path, 'camera_extrinsic_matrix.npy'))
camera_ext_matrix

array([[ 0.7071068 ,  0.        ,  0.7071068 ,  0.        ],
       [ 0.66446304,  0.34202015, -0.66446304, -0.04894316],
       [-0.24184477,  0.9396926 ,  0.24184477, -5.3030753 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [46]:
global_scale = 1
obs = raw_obs
depth = obs[..., -1] / global_scale
color = obs[..., :3][..., ::-1] / global_scale

In [47]:
fgpcd = depth2fgpcd(depth, camera_intrinsic_params, camera_ext_matrix)
# fgpcd = downsample_pcd(fgpcd, 0.01)
# fgpcd = depth2fgpcd_top(depth, depth<0.599/0.8, camera_intrinsic_params)
print(depth.shape)
print(color.shape)
print(fgpcd.shape)

(720, 720)
(720, 720, 3)
(46414, 3)


In [48]:
pcd = o3d.geometry.PointCloud()

fgpcd = fgpcd[..., [0, 2, 1]]
fgpcd[..., 2] = -fgpcd[..., 2]

pcd.points = o3d.utility.Vector3dVector(fgpcd)
o3d.visualization.draw_geometries([pcd])

In [49]:
# save to pcd file
o3d.io.write_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'), pcd)

True

In [50]:
# save pcd picture 
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.get_render_option().point_color_option = o3d.visualization.PointColorOption.Color
# vis.get_render_option().point_size = 3.0
# vis.add_geometry(pcd)
# vis.capture_screen_image(str(view) + ".jpg", do_render=True)
# vis.destroy_window()